# k 폴드 교차검증
- 데이터 량이 적거나 데이터가 편차등으로 인해 일정한수준의 머신러닝 효율을 보이지 않을때
    - 머신러닝의 경우 해당트레이닝 세트에 최적화된 결과만 도출-> 과적합의 위험
    - 트레이닝세트를 변화시켜 과적합의 위험을 줄이고 공정한 판단
- 데이터셋을 n개로 나누고 블럭별로 돌아가면서 타겟값으로 지정하여 분석
- 최종적으로 블럭별 나뉘어진 결과의 평균으로 머신러닝의 효율을 측정
- 장점
    * 모든 데이터셋을 활용
    * 데이터 부족에 따른 편차 제거
    * 일반화된 평가 모델(검증)
- 단점
    * 계산량이 많아서 비용 발생( 평가시간 오래걸림)
<br>
<img src="img/kfold.png" width=400/>

In [26]:
# 라이브러리 불러오기
import sklearn
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

In [27]:
# 데이터셋 수집
from sklearn.datasets import load_iris
iris=load_iris()
iris.keys()

dict_keys(['data', 'target', 'frame', 'target_names', 'DESCR', 'feature_names', 'filename'])

* 데이터 성격 파악
    -Attribute Information:
        - sepal length in cm
        - sepal width in cm
        - petal length in cm
        - petal width in cm
        - class:
                - Iris-Setosa
                - Iris-Versicolour
                - Iris-Virginica

In [30]:
# 데이터 준비 
# 데이터 합치기
# 기본적 데이터 확인
cols=['slen','swid','plen','pwid']
idf=pd.DataFrame(data,columns=cols)
idf['target']=iris['target']
idf.tail()

,slen,swid,plen,pwid,target
145,6.7,3.0,5.2,2.3,2
146,6.3,2.5,5.0,1.9,2
147,6.5,3.0,5.2,2.0,2
148,6.2,3.4,5.4,2.3,2
149,5.9,3.0,5.1,1.8,2


In [60]:
# 데이터 준비
from sklearn.model_selection import KFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

kfold=KFold(n_splits=5)
dtc=DecisionTreeClassifier(random_state=123)
data=idf.loc[:,['slen','swid','plen','pwid']].values
accs=[]
for train_i,test_i in kfold.split(idf):
    X_train,X_test=data[train_i],data[test_i]
    y_train,y_test=iris['target'][train_i],iris['target'][test_i]
    #print('*'*30)
    #print(y_test)
    dtc.fit(X_train,y_train)
    pred=dtc.predict(X_test)
    acco=accuracy_score(pred,y_test)
    acc=np.round(acco,4)
    accs.append(acc)
kfoldacc=np.mean(accs)
kacc=np.round(kfoldacc,4)
print('k폴드정확도:',kacc)

k폴드정확도: 0.92


#### Stratified Kfold
* 기존 K 폴드가 편향이 있는 데이터의 경우 폴드수가 작으면 심각한 검증 오류 발생
* 기존 원본비율을 유지하면서 다층으로 분석하는 형태의 kfold


In [61]:
# Stratified Kfold : 원본 비율유지 k 폴드
from sklearn.model_selection import StratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

skfold=StratifiedKFold(n_splits=5)
dtc=DecisionTreeClassifier(random_state=123)
data=idf.loc[:,['slen','swid','plen','pwid']].values
accs=[]
for train_i,test_i in skfold.split(data,iris['target']):
    X_train,X_test=data[train_i],data[test_i]
    y_train,y_test=iris['target'][train_i],iris['target'][test_i]
    #print('*'*30)
    #print(y_test,test_i)
    dtc.fit(X_train,y_train)
    pred=dtc.predict(X_test)
    acco=accuracy_score(pred,y_test)
    acc=np.round(acco,4)
    accs.append(acc)
skfoldacc=np.mean(accs)
skacc=np.round(skfoldacc,4)
print('k폴드정확도:',skacc)

k폴드정확도: 0.96


In [18]:
# 모델을 평가
pred=dtc.predict(X_test)
pred

array([1, 2, 2, 1, 0, 1, 1, 0, 0, 1, 2, 0, 1, 2, 2, 2, 0, 0, 1, 0, 0, 2,
       0, 2, 0, 0, 0, 2, 2, 0])

In [19]:
y_test

array([1, 2, 2, 1, 0, 2, 1, 0, 0, 1, 2, 0, 1, 2, 2, 2, 0, 0, 1, 0, 0, 2,
       0, 2, 0, 0, 0, 2, 2, 0])

In [20]:
# 라이브러리 안쓰고 정밀도 판단
predf=pd.DataFrame(pred,columns=['pred'])
predf['label']=y_test
predf['diff']=predf['label']-predf['pred']
base=len(predf)
cor=len(predf[predf['diff']!=0])
cor/base

0.03333333333333333

In [21]:
# 라이브러리 써서 판단

acc=accuracy_score(pred,y_test)
acc

0.9666666666666667

In [22]:
def dtcPrs(X_train,X_test,y_train,y_test):
    
    dtc=DecisionTreeClassifier() # 모델설정
    dtc.fit(X_train,y_train) # 트레이닝
    
    pred=dtc.predict(X_test) # 테스트 예측값 생성
    acc=accuracy_score(pred,y_test)
    print('명중률:',acc)
dtcPrs(X_train,X_test,y_train,y_test) 

명중률: 0.9666666666666667


In [23]:
def mldtcPrs(X,Y):
    X_train,X_test,y_train,y_test=train_test_split(X,Y,shuffle=True,test_size=0.2)
    dtcPrs(X_train,X_test,y_train,y_test)
mldtcPrs(X,Y)

명중률: 1.0


In [24]:
# knn 모델 사용
from sklearn.neighbors import KNeighborsClassifier
def knnPrs(X_train,X_test,y_train,y_test,param=2):
    knn=KNeighborsClassifier(n_neighbors=param) # 모델설정
    knn.fit(X_train,y_train) # 트레이닝
    # 평가
    pred=knn.predict(X_test) # 테스트 예측값 생성
    acc=accuracy_score(pred,y_test)
    print('knn 명중률:',acc)
    return acc
knnPrs(X_train,X_test,y_train,y_test) 

knn 명중률: 0.9


0.9

In [25]:
# 예측모델 : 모델선정, 파라미터값